In [1]:
import pandas as pd
import numpy as np
import fiona
import matplotlib.pyplot as plt
import rtree
import string
import pyproj
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon, shape
from shapely.prepared import prep
from descartes import PolygonPatch
from itertools import chain
from ast import literal_eval
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, GridSearchCV
#%matplotlib inline

In [2]:
# set ipython's max row display
pd.set_option('display.max_row', 3)

# set iPython's max column width to 50
pd.set_option('display.max_columns', 55)

In [7]:
#pad the map
shp = fiona.open('shapefiles/my_detroit.shp')
bds = shp.bounds
shp.close()
padding = 0.01
ll = (bds[0], bds[1])
ur = (bds[2], bds[3])
coords = list(chain(ll, ur))
w, h = coords[2] - coords[0], coords[3] - coords[1]

In [ ]:
m = Basemap(
    projection='merc',
    lon_0=bds[0] + w / 2.,
    lat_0=bds[1] + h / 2.,
    ellps = 'WGS84',
    llcrnrlon=coords[0] - padding * w,
    llcrnrlat=coords[1] - .03,
    urcrnrlon=coords[2] + padding * w,
    urcrnrlat=coords[3] + .005,
    lat_ts=0,
    resolution='c',
    suppress_ticks=True);

m.readshapefile(
    'shapefiles/my_detroit',
    'parcels');

#m.readshapefile(
#    'detroit_census',
#    'tracts');

In [5]:
#read in shapefile to pd.DataFrame
df_detroit = pd.DataFrame({
        'poly': [Polygon(xy) for xy in m.parcels],
        'parcel_no': [parcel['PARCELNO'] for parcel in m.parcels_info],
        'address': [parcel['PROPADDR'] for parcel in m.parcels_info],
        'sale_value': [parcel['SEV'] for parcel in m.parcels_info],
        'tax_status': [parcel['TAXSTATUS'] for parcel in m.parcels_info]
    })

#get rid of dummy addresses
df_detroit = df_detroit[df_detroit['address'] != df_detroit['address'][0]]

In [6]:
m.parcels_info[100].keys()

dict_keys(['SEV', 'DISTRICT', 'CIBFLAREA', 'ISIMPROVED', 'LANDMAP', 'TOTALSQFT', 'PARCELNO', 'XSTRNAME2', 'RESYRBUILT', 'CIBBLDGNO', 'TAXPAYER2', 'XSTRNAME1', 'OBJECTID', 'HOMESTEAD', 'AV', 'LEGAL', 'ECF', 'RINGNUM', 'RP', 'CIBOCC', 'SUBDIVISIO', 'TAXPAYER1', 'FRONTAGE', 'PROPADDR', 'PREVTSTATU', 'PROPZIP', 'SALEDATE', 'RELATED', 'Shape_STAr', 'PREVPCLASS', 'TAXZIP', 'SALEPRICE', 'SHAPENUM', 'SPECACT', 'TAXCITY', 'PROPSTATUS', 'TV', 'PROPSTR', 'DEPTH', 'NEZ', 'TAXSTATE', 'TOTALACREA', 'TAXSTATUS', 'Shape_STLe', 'PROPNO', 'LANDVALUE', 'RESSTYLE', 'RESBLDGNO', 'USECODE', 'CIBYRBUILT', 'RESOCC', 'RESFLAREA', 'PROPDIR', 'AKA', 'PROPCLASS', 'WARD', 'TAXADDR'])

In [7]:
#def remove_punc(x):
#    return x.translate(str.maketrans({key: None for key in string.punctuation}))

In [8]:
#df_detroit['tax_status'] = df_detroit['tax_status'].apply(remove_punc)

In [9]:
parcels = df_detroit['parcel_no'].unique()

In [10]:
parcel_points = pd.read_csv('parcel_points_ownership.csv', low_memory=False)

In [11]:
parcel_points = parcel_points[['ParcelNo', 'Latitude', 'Longitude']]

In [12]:
df_detroit = df_detroit.merge(parcel_points,
                              how='left',
                              left_on='parcel_no',
                              right_on='ParcelNo'
                             ).fillna(-1, downcast='infer')

In [13]:
dupe_idx = df_detroit[df_detroit.drop('poly', axis=1).duplicated()].index

In [14]:
df_detroit.drop(dupe_idx, inplace=True)

In [15]:
df_detroit['tax_status'].unique()

array(['TAXABLE', 'COUNTY OWNED', 'CITY OWNED', 'RELIGIOUS', 'STATUTORY',
       'EDUCATION/SCIENTIFIC', 'STATE LAND BANK', 'REAL ESTATE DIVISION',
       'CITY LAND BANK', 'BENEVOLENT/CHARITY', 'PUBLIC LIGHTING',
       'STATE OWNED', 'NON-PROFIT', 'PILOT', 'DETROIT HOUSING COMM',
       'COMMUNICATIONS', 'BOARD OF EDUCATION', 'RAILROAD', 'HOSPITAL',
       'PARSONAGE', 'FEDERAL', 'WATER DEPARTMENT', 'PA 245',
       'DISABLED VETERAN', 'PROJECTS', 'COUNTY LAND BANK',
       '2YR NON-PROFIT HOUSE', 'SEMTA'], dtype=object)

In [16]:
df_detroit = pd.concat([df_detroit, pd.get_dummies(df_detroit['tax_status'])], axis=1)

In [17]:
df_detroit

,address,parcel_no,poly,sale_value,tax_status,ParcelNo,Latitude,Longitude,2YR NON-PROFIT HOUSE,BENEVOLENT/CHARITY,BOARD OF EDUCATION,CITY LAND BANK,CITY OWNED,COMMUNICATIONS,COUNTY LAND BANK,COUNTY OWNED,DETROIT HOUSING COMM,DISABLED VETERAN,EDUCATION/SCIENTIFIC,FEDERAL,HOSPITAL,NON-PROFIT,PA 245,PARSONAGE,PILOT,PROJECTS,PUBLIC LIGHTING,RAILROAD,REAL ESTATE DIVISION,RELIGIOUS,SEMTA,STATE LAND BANK,STATE OWNED,STATUTORY,TAXABLE,WATER DEPARTMENT
0,400 E JEFFERSON,01000001.,"POLYGON ((27802.8046927004 15612.52909134701, ...",55094700,TAXABLE,01000001.,42.329092,-83.03981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385175,14200 W OUTER DRIVE,22126050-1,"POLYGON ((3341.801573943281 20939.47687371168,...",14400,TAXABLE,22126050-1,42.364941,-83.26117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
#for item in list(df_detroit['tax_status'].unique()):
#    df_detroit[item] = 0
#    df_detroit.loc[df_detroit['tax_status'] == item, item] = 1

In [19]:
del df_detroit['ParcelNo']

In [20]:
del df_detroit['tax_status']

In [21]:
df_detroit

,address,parcel_no,poly,sale_value,Latitude,Longitude,2YR NON-PROFIT HOUSE,BENEVOLENT/CHARITY,BOARD OF EDUCATION,CITY LAND BANK,CITY OWNED,COMMUNICATIONS,COUNTY LAND BANK,COUNTY OWNED,DETROIT HOUSING COMM,DISABLED VETERAN,EDUCATION/SCIENTIFIC,FEDERAL,HOSPITAL,NON-PROFIT,PA 245,PARSONAGE,PILOT,PROJECTS,PUBLIC LIGHTING,RAILROAD,REAL ESTATE DIVISION,RELIGIOUS,SEMTA,STATE LAND BANK,STATE OWNED,STATUTORY,TAXABLE,WATER DEPARTMENT
0,400 E JEFFERSON,01000001.,"POLYGON ((27802.8046927004 15612.52909134701, ...",55094700,42.329092,-83.03981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385175,14200 W OUTER DRIVE,22126050-1,"POLYGON ((3341.801573943281 20939.47687371168,...",14400,42.364941,-83.26117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Census

In [22]:
#didn't help and the added features slow down runtime

#df_census = pd.DataFrame({
#        'census_poly': [Polygon(xy) for xy in m.tracts],
#        'tract_id': [tract['NAME10'] for tract in m.tracts_info]
#        })

In [23]:
#indexed_census = zip(df_census['census_poly'].index.tolist(), df_census['census_poly'])
#idx = rtree.index.Index()
#for index, poly in indexed_census:
#    idx.insert(index, poly.bounds)

In [24]:
#df_detroit['coordinates'] = list(zip(df_detroit['Longitude'], df_detroit['Latitude']))
#df_detroit['poly_idx'] = df_detroit['coordinates'].apply(
#    lambda x: find_intersecting_polys(df_census['census_poly'], idx, m, x))

In [25]:
#df_detroit = df_detroit.merge(df_census,
#                how='left',
#                left_on='poly_idx',
#                right_index=True
#                )

In [26]:
#del df_detroit['coordinates']
#del df_detroit['poly_idx']
#del df_detroit['census_poly']

In [27]:
#df_detroit.dropna(inplace=True)

In [28]:
#df_detroit = pd.concat([df_detroit, pd.get_dummies(df_detroit['tract_id'])], axis=1)

In [29]:
#del df_detroit['tract_id']

In [30]:
#df_detroit

# Blight

In [31]:
df_blight = pd.read_csv('detroit-blight-violations.csv', low_memory=False)

In [32]:
df_blight = df_blight[df_blight['ParcelNo'].isin(parcels)]

In [33]:
df_blight = df_blight[['ViolationCode', 'ParcelNo']]

In [34]:
#df_blight['ViolationCode'] = df_blight['ViolationCode'].apply(remove_punc)

In [35]:
for item in list(df_blight['ViolationCode'].unique()):
    df_blight[item] = 0
    df_blight.loc[df_blight['ViolationCode'] == item, item] = 1

In [36]:
del df_blight['ViolationCode']

In [37]:
df_blight

,ParcelNo,22-2-88(b),9-1-104,9-1-110(a),22-2-88(a),22-2-43,9-1-113,9-1-202,61-85.0100/32.0066,9-1-83 - (Structures < 5 stories),9-1-81(a),9-1-36(a),9-1-82(d) - (Structures < 5 stories),9-1-82(d) - (Dwelling),9-1-43(a) - (Dwelling),9-1-43(a) - (Structures),9-1-13,9-1-111,22-2-45,22-2-44,61-5-14 (9),9-1-107,9-1-111(a),9-1-50(a),61-85.0100/32.0031,22-2-16,22-2-17(a),...,61-84.0100/32.0066,22-2-18,61-8-27,9-1-311,22-2-97,61-4-37,9-1-43(a) - (Stories),9-1-103 (a) or (b),61-101.0100/32.0066,61-63.0600,9-1-102,22-2-87,22-2-84(b)(3),61-5-20,22-2-23,22-2-88(c),9-1-221,61-86.0100/32.0066,9-1-42,22-2-96,9-1-208,61-8-127,9-1-83,61-130.0000/130.0500,61-115.0100/32.0066,9-1-475,22-2-84(a)(1)(2)(4)
0,22108345.,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20285,15013804.,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
df_blight.groupby('ParcelNo').sum()

,22-2-88(b),9-1-104,9-1-110(a),22-2-88(a),22-2-43,9-1-113,9-1-202,61-85.0100/32.0066,9-1-83 - (Structures < 5 stories),9-1-81(a),9-1-36(a),9-1-82(d) - (Structures < 5 stories),9-1-82(d) - (Dwelling),9-1-43(a) - (Dwelling),9-1-43(a) - (Structures),9-1-13,9-1-111,22-2-45,22-2-44,61-5-14 (9),9-1-107,9-1-111(a),9-1-50(a),61-85.0100/32.0031,22-2-16,22-2-17(a),22-2-49(a),...,61-84.0100/32.0066,22-2-18,61-8-27,9-1-311,22-2-97,61-4-37,9-1-43(a) - (Stories),9-1-103 (a) or (b),61-101.0100/32.0066,61-63.0600,9-1-102,22-2-87,22-2-84(b)(3),61-5-20,22-2-23,22-2-88(c),9-1-221,61-86.0100/32.0066,9-1-42,22-2-96,9-1-208,61-8-127,9-1-83,61-130.0000/130.0500,61-115.0100/32.0066,9-1-475,22-2-84(a)(1)(2)(4)
ParcelNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01000122.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22126046.,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
df_detroit = df_detroit.merge(df_blight.groupby('ParcelNo').sum(),
                              how='left',
                              left_on='parcel_no',
                              right_index=True
                             ).fillna(-1, downcast='infer')

In [40]:
df_detroit

,address,parcel_no,poly,sale_value,Latitude,Longitude,2YR NON-PROFIT HOUSE,BENEVOLENT/CHARITY,BOARD OF EDUCATION,CITY LAND BANK,CITY OWNED,COMMUNICATIONS,COUNTY LAND BANK,COUNTY OWNED,DETROIT HOUSING COMM,DISABLED VETERAN,EDUCATION/SCIENTIFIC,FEDERAL,HOSPITAL,NON-PROFIT,PA 245,PARSONAGE,PILOT,PROJECTS,PUBLIC LIGHTING,RAILROAD,REAL ESTATE DIVISION,...,61-84.0100/32.0066,22-2-18,61-8-27,9-1-311,22-2-97,61-4-37,9-1-43(a) - (Stories),9-1-103 (a) or (b),61-101.0100/32.0066,61-63.0600,9-1-102,22-2-87,22-2-84(b)(3),61-5-20,22-2-23,22-2-88(c),9-1-221,61-86.0100/32.0066,9-1-42,22-2-96,9-1-208,61-8-127,9-1-83,61-130.0000/130.0500,61-115.0100/32.0066,9-1-475,22-2-84(a)(1)(2)(4)
0,400 E JEFFERSON,01000001.,"POLYGON ((27802.8046927004 15612.52909134701, ...",55094700,42.329092,-83.03981,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385175,14200 W OUTER DRIVE,22126050-1,"POLYGON ((3341.801573943281 20939.47687371168,...",14400,42.364941,-83.26117,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


# 311 Violations

In [41]:
indexed_polys = zip(df_detroit['poly'].index.tolist(), df_detroit['poly'])
idx = rtree.index.Index()
for index, poly in indexed_polys:
    idx.insert(index, poly.bounds)

In [42]:
def find_intersecting_polys(polygons, poly_tree, my_map, coordinates):
    '''returns points within a polygon using rtree given:
       pd.Series of polygons, rtree index, Basemap object,
       and tuple of long,lat coordinates'''
    #convert points to map coordinates
    point = Point(my_map(coordinates[0], coordinates[1]))
    poly_idx = [poly_index for poly_index in poly_tree.intersection(point.coords[0])
                if point.within(polygons.ix[poly_index])]
    if poly_idx:
        return poly_idx[0]
    else:
        return np.nan

In [43]:
df_311 = pd.read_csv('detroit-311.csv')
df_311['coordinates'] = list(zip(df_311['lng'], df_311['lat']))

In [44]:
df_311 = df_311[['issue_type', 'coordinates']]

In [45]:
#df_311['issue_type'] = df_311['issue_type'].apply(remove_punc)

In [46]:
for item in list(df_311['issue_type'].unique()):
    df_311[item] = 0
    df_311.loc[df_311['issue_type'] == item, item] = 1

In [47]:
del df_311['issue_type']

In [48]:
df_311 = df_311.groupby('coordinates', as_index=False).sum()

In [49]:
df_311['poly_idx'] = df_311['coordinates'].apply(
    lambda x: find_intersecting_polys(df_detroit['poly'], idx, m, x))

In [50]:
del df_311['coordinates']

In [51]:
df_311.dropna(inplace=True)

In [52]:
df_311['poly_idx'] = df_311['poly_idx'].astype(int) 

In [53]:
df_311

,Traffic Signal Issue,Blocked Catch Basin,Fire Hydrant Issue,Potholes,Tree Issue,Running Water in a Home or Building,Illegal Dumping / Illegal Dump Sites,Manhole Cover Issue,Traffic Sign Issue,DPW - Other environmental,Water Main Break,"Graffiti Abatement (internal use only, public issue)",Street Light Pole Down,Detroit Land Bank Referral,Curbside Solid Waste Issue,Abandoned Vehicle,DPW - Debris Removal - DPW USE ONLY,Graffiti,Dead Animal Removal,Trash Issue - Refuse container left at curbside between collection days,Residential Snow Removal Issue,Trash Issue - Bulk waste deposited more than 24 hours before designated time,poly_idx
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,383958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,275145


In [54]:
df_detroit = df_detroit.merge(df_311.groupby('poly_idx').sum(),
                              how='left',
                              left_index=True,
                              right_index=True
                             ).fillna(-1, downcast='infer')

In [55]:
df_detroit

,address,parcel_no,poly,sale_value,Latitude,Longitude,2YR NON-PROFIT HOUSE,BENEVOLENT/CHARITY,BOARD OF EDUCATION,CITY LAND BANK,CITY OWNED,COMMUNICATIONS,COUNTY LAND BANK,COUNTY OWNED,DETROIT HOUSING COMM,DISABLED VETERAN,EDUCATION/SCIENTIFIC,FEDERAL,HOSPITAL,NON-PROFIT,PA 245,PARSONAGE,PILOT,PROJECTS,PUBLIC LIGHTING,RAILROAD,REAL ESTATE DIVISION,...,9-1-83,61-130.0000/130.0500,61-115.0100/32.0066,9-1-475,22-2-84(a)(1)(2)(4),Traffic Signal Issue,Blocked Catch Basin,Fire Hydrant Issue,Potholes,Tree Issue,Running Water in a Home or Building,Illegal Dumping / Illegal Dump Sites,Manhole Cover Issue,Traffic Sign Issue,DPW - Other environmental,Water Main Break,"Graffiti Abatement (internal use only, public issue)",Street Light Pole Down,Detroit Land Bank Referral,Curbside Solid Waste Issue,Abandoned Vehicle,DPW - Debris Removal - DPW USE ONLY,Graffiti,Dead Animal Removal,Trash Issue - Refuse container left at curbside between collection days,Residential Snow Removal Issue,Trash Issue - Bulk waste deposited more than 24 hours before designated time
0,400 E JEFFERSON,01000001.,"POLYGON ((27802.8046927004 15612.52909134701, ...",55094700,42.329092,-83.03981,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385175,14200 W OUTER DRIVE,22126050-1,"POLYGON ((3341.801573943281 20939.47687371168,...",14400,42.364941,-83.26117,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


# Fire

In [56]:
df_fire = pd.read_csv('detroit-fire.csv')

In [57]:
#remove entrys with slashes they contain no street number
df_fire.loc[:, 'ADDRESS'] = df_fire[~df_fire['ADDRESS'].str.contains('/')]

df_fire = df_fire[['INCIDENT TYPE', 'ADDRESS']]
df_fire.dropna(inplace=True)

#uppercase and remove trailing detroit
df_fire.loc[:, 'ADDRESS'] = df_fire['ADDRESS'].str.split(',').str[0].str.upper()

#remove unit number
df_fire.loc[:, 'ADDRESS'] = df_fire['ADDRESS'].str.replace(r'UNIT#\s*\d*', '')

In [58]:
df_fire['ADDRESS']

1             18321 FAUST AVE
                 ...         
19995    1777 N RADEMACHER ST
Name: ADDRESS, dtype: object

In [59]:
df_fire = df_fire[['INCIDENT TYPE', 'ADDRESS']]

In [60]:
#df_fire['INCIDENT TYPE'] = df_fire['INCIDENT TYPE'].apply(remove_punc)

In [61]:
#create dictionary of standard postal abbreviations scraped from USPS website
st_abbv = pd.read_csv('postal_abbrevs2.csv').fillna(method='pad')
st_abbv_map = dict(zip(st_abbv['common_suffix'], st_abbv['standard_suffix']))

In [62]:
def find_duplicate_street_names(street_series, dupes=set()):
    '''accepts pd.Series of street names and returns street names that truncation would make the same'''
    directions = set(['N', 'NW', 'NE', 'E', 'W', 'SW', 'SE', 'S'])
    names = sorted(list(set(map(lambda x: tuple(x.translate(str.maketrans({key: None for key in string.punctuation})).split())[1:], street_series))))

    streets = [list(name) for name in names if len(name) > 1 and name[-1] in st_abbv_map]
    for i in range(len(streets)):
        # standardize name
        if len(streets[i]) == 2 and streets[i][0] in directions:
            pass
        else:
            streets[i][-1] = st_abbv_map[streets[i][-1]]
        # don't flag repeats of same standardized name or different numbers of words
        if streets[i] == streets[i-1] or len(streets[i]) != len(streets[i-1]):
            pass
        # flag if standardized names are equal
        elif streets[i][0:-1] == streets[i-1][0:-1]:
            dupes.add(tuple(streets[i-1]))
            dupes.add(tuple(streets[i]))
        
    return dupes

In [63]:
duplicate_streets = find_duplicate_street_names(df_detroit['address'].unique())

In [64]:
duplicate_streets = find_duplicate_street_names(df_fire['ADDRESS'].unique(), dupes=duplicate_streets)

In [65]:
def clean_site_address(address, dupes):
    '''for SITE_ADDRESS detroit-demolition-permits.tsv'''
    #tmp = address.translate(None, string.punctuation).split()
    directions = set(['N', 'NW', 'NE', 'E', 'W', 'SW', 'SE', 'S'])
    tmp = address.split()
    for i in range(1, len(tmp)):
        tmp[i] = tmp[i].upper().translate(str.maketrans({key: None for key in string.punctuation}))
    if len(tmp) == 2:
        pass
    else:
        # standardize name
        if len(tmp) == 3 and set(tmp[i]).intersection(directions):
            pass
        elif tmp[-1] in st_abbv_map:
            tmp[-1] = st_abbv_map[tmp[-1]]
        # exclude street number and check whether duplicate and last word is a street abbreviation
        if tuple(tmp[1:]) not in dupes and tmp[-1] in set(st_abbv['standard_suffix']):
            del tmp[-1]
    return " ".join(tmp)

In [66]:
df_fire['ADDRESS'].dropna(inplace=True)

In [67]:
df_fire['ADDRESS'] = df_fire['ADDRESS'].apply(lambda x: clean_site_address(x, duplicate_streets))

In [68]:
for item in list(df_fire['INCIDENT TYPE'].unique()):
    df_fire[item] = 0
    df_fire.loc[df_fire['INCIDENT TYPE'] == item, item] = 1

In [69]:
df_fire.groupby('ADDRESS').sum()

,745 - unintentional,"710 - Malicious, mischievous false call, other",442 - Overheated motor,"700 - False alarm or false call, other","160 - Special outside fire, other","651 - Smoke scare, odor of smoke","650 - Steam, other gas mistaken for smoke, other","440 - Electrical wiring/equipment problem, other","730 - System malfunction, other",143 - Grass fire,561 - Unauthorized burning,"240 - Explosion (no fire), other","100 - Fire, other",412 - Gas leak (natural gas or LPG),"220 - Overpressure rupture from air or gas, other",551 - Assist police or other governmental agency,"420 - Toxic condition, other",522 - Water or steam leak,444 - Power line down,"118 - Trash or rubbish fire, contained",111 - Building fire,"600 - Good intent call, other","130 - Mobile property (vehicle) fire, other",411 - Gasoline or other flammable liquid spill,733 - Smoke detector activation due to malfunction,"150 - Outside rubbish fire, other","652 - Steam, vapor, fog or dust thought to be smoke",...,461 - Building or structure weakened or collapsed,814 - Lightning strike (no fire),521 - Water evacuation,133 - Rail vehicle fire,90 - Special type of incident,422 - Chemical spill or leak,221 - Overpressure rupture of air or gas pipe/pipeline,74 - no fire,815 - Severe weather or natural disaster standby,632 - Prescribed fire,"813 - Wind storm, tornado/hurricane assessment","117 - Commercial Compactor fire, confined to rubbish","70 - False alarm and false call, other",911 - Citizen complaint,161 - Outside storage fire,134 - Water vehicle fire,"540 - Animal problem, other",413 - Oil or other combustible liquid spill,243 - Fireworks explosion (no fire),135 - Aircraft fire,"800 - Severe weather or natural disaster, other",72 - Bomb scare,"451 - Biological hazard, confirmed or suspected",137 - Camper or recreational vehicle (RV) fire,"116 - Fuel burner/boiler malfunction, fire confined","550 - Public service assistance, other",132 - Road freight or transport vehicle fire
ADDRESS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1 CASINO WAY,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S 541 PARK,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [70]:
df_detroit['address'] = df_detroit['address'].apply(lambda x: clean_site_address(x, duplicate_streets))

In [71]:
df_detroit = df_detroit.merge(df_fire.groupby('ADDRESS').sum(),
                              how='left',
                              right_index=True,
                              left_on='address'
                             ).fillna(-1, downcast='infer')

In [72]:
df_detroit

,address,parcel_no,poly,sale_value,Latitude,Longitude,2YR NON-PROFIT HOUSE,BENEVOLENT/CHARITY,BOARD OF EDUCATION,CITY LAND BANK,CITY OWNED,COMMUNICATIONS,COUNTY LAND BANK,COUNTY OWNED,DETROIT HOUSING COMM,DISABLED VETERAN,EDUCATION/SCIENTIFIC,FEDERAL,HOSPITAL,NON-PROFIT,PA 245,PARSONAGE,PILOT,PROJECTS,PUBLIC LIGHTING,RAILROAD,REAL ESTATE DIVISION,...,461 - Building or structure weakened or collapsed,814 - Lightning strike (no fire),521 - Water evacuation,133 - Rail vehicle fire,90 - Special type of incident,422 - Chemical spill or leak,221 - Overpressure rupture of air or gas pipe/pipeline,74 - no fire,815 - Severe weather or natural disaster standby,632 - Prescribed fire,"813 - Wind storm, tornado/hurricane assessment","117 - Commercial Compactor fire, confined to rubbish","70 - False alarm and false call, other",911 - Citizen complaint,161 - Outside storage fire,134 - Water vehicle fire,"540 - Animal problem, other",413 - Oil or other combustible liquid spill,243 - Fireworks explosion (no fire),135 - Aircraft fire,"800 - Severe weather or natural disaster, other",72 - Bomb scare,"451 - Biological hazard, confirmed or suspected",137 - Camper or recreational vehicle (RV) fire,"116 - Fuel burner/boiler malfunction, fire confined","550 - Public service assistance, other",132 - Road freight or transport vehicle fire
0,400 E JEFFERSON,01000001.,"POLYGON ((27802.8046927004 15612.52909134701, ...",55094700,42.329092,-83.03981,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385175,14200 W OUTER,22126050-1,"POLYGON ((3341.801573943281 20939.47687371168,...",14400,42.364941,-83.26117,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


# Demolitions

In [73]:
df_demos = pd.read_csv('detroit-demolition-permits.csv')

In [74]:
df_demos = df_demos[df_demos['Parcel ID'].isin(parcels)]

In [75]:
df_demos

,Address,Parcel ID,Contractor Name,Price,Demolition Date,Commercial Building,Council_District,Latitude,Longitude,Location
0,1319 Green,18009074.,Homrich,$11200.00,02/10/2016,No,6,42.306876,-83.116669,"1319 Green\n(42.306876, -83.116669)"
...,...,...,...,...,...,...,...,...,...,...
7111,11731 Beaverland,22118707.,Able Demolition,$7400.00,01/25/2016,No,7,42.373504,-83.264960,"11731 Beaverland\n(42.373504, -83.26496)"


In [76]:
len(df_demos.groupby('Parcel ID'))

7110

In [77]:
df_demos = df_demos['Parcel ID'].to_frame()
df_demos['is_blighted'] = 1

In [78]:
df_detroit = df_detroit.merge(df_demos,
                              how='left',
                              right_on='Parcel ID',
                              left_on='parcel_no'
                             ).fillna(0, downcast='infer')

In [79]:
del df_detroit['Parcel ID']

# Exporting

In [80]:
df_detroit.dropna(inplace=True)

In [81]:
df_detroit.drop_duplicates('parcel_no', inplace=True)

In [82]:
df_detroit.to_csv('all_data.csv', index=False)